In [ ]:
!pip3 install torch
!pip3 install dgl==0.4.3post2 
!pip3 install dglke
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '/Users/ravinschmidl/Desktop/CoV-KGE-master/utils')
drkg_file = '/Users/ravinschmidl/Desktop/CoV-KGE-master/KG.tsv'
df = pd.read_csv(drkg_file, sep="\t", index_col = 0)


In [ ]:
df = df.drop(['r_name'], axis = 1)

In [ ]:
triples = df.values.tolist()


In [ ]:
 num_triples

In [ ]:
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

with open("/Users/ravinschmidl/Desktop/CoV-KGE-master/train/kg_train.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("/Users/ravinschmidl/Desktop/CoV-KGE-master/train/kg_valid.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("/Users/ravinschmidl/Desktop/CoV-KGE-master/train/kg_test.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

In [ ]:
!DGLBACKEND=pytorch dglke_train --dataset KG --data_path /Users/ravinschmidl/Desktop/CoV-KGE-master/train --data_files kg_train.tsv kg_valid.tsv kg_test.tsv --format 'raw_udd_hrt' --model_name RotatE --batch_size 1024 \
--neg_sample_size -256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Reading train triples....
Finished. Read 9615358 train triples.
Reading valid triples....
Finished. Read 534186 valid triples.
Reading test triples....
Finished. Read 534188 test triples.
|Train|: 9615358
random partition 9615358 edges into 8 parts
part 0 has 1201920 edges
part 1 has 1201920 edges
part 2 has 1201920 edges
part 3 has 1201920 edges
part 4 has 1201920 edges
part 5 has 1201920 edges
part 6 has 1201920 edges
part 7 has 1201918 edges
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 534186
|test|: 534188
Total initialize time 40.682 seconds
[W ParallelNative.cpp:206] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:206] Warning: Can

In [ ]:
!DGLBACKEND=pytorch dglke_train --dataset KG --data_path /Users/ravinschmidl/Desktop/CoV-KGE-master/train --data_files kg_train.tsv kg_valid.tsv kg_test.tsv --format 'raw_udd_hrt' --model_name RotatE --batch_size 1024 \
--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update
